# Coding Challenge Skynopy Mission Manager

This is the strucure code to help you respond to the coding challenge

In [2]:
import pandas as pd


from ostk.mathematics.object import RealInterval

from ostk.physics import Environment
from ostk.physics.unit import Length
from ostk.physics.unit import Angle
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.time import Time
from ostk.physics.coordinate.spherical import LLA
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.trajectory.orbit.model import Kepler
from ostk.astrodynamics.trajectory.orbit.model.kepler import COE
from ostk.astrodynamics.trajectory.orbit.model import SGP4
from ostk.astrodynamics.trajectory.orbit.model.sgp4 import TLE
from ostk.astrodynamics.access import Generator as AccessGenerator
from ostk.astrodynamics.access import AccessTarget
from ostk.astrodynamics.access import VisibilityCriterion
from ostk.astrodynamics.utilities import compute_trajectory_geometry
from ostk.astrodynamics.utilities import compute_time_lla_aer_coordinates

In [3]:

environment = Environment.default(True)  # Set global environment instance as the default instance 
earth = environment.access_celestial_object_with_name("Earth")

In [4]:
visibility_criterion = VisibilityCriterion.from_aer_interval(
    azimuth_interval=RealInterval.closed(0.0, 360.0),  # [deg]
    elevation_interval=RealInterval.closed(5.0, 90.0),  # [deg]
    range_interval=RealInterval.closed(0.0, 10000e3),  # [m]
)

In [5]:
access_target = AccessTarget.from_lla(
    visibility_criterion=visibility_criterion,
    lla=LLA(Angle.degrees(50.0), Angle.degrees(20.0), Length.meters(30.0)),
    celestial=earth,
)
another_access_target = AccessTarget.from_lla(
    visibility_criterion=visibility_criterion,
    lla=LLA(Angle.degrees(-50.0), Angle.degrees(10.0), Length.meters(30.0)),
    celestial=earth,
)

access_targets = [access_target, another_access_target]

In [14]:
epoch = Instant.date_time(DateTime(2018, 1, 1, 0, 0, 0), Scale.UTC)

satellite_orbit = Orbit.sun_synchronous(
    epoch, Length.kilometers(500.0), Time(12, 0, 0), earth
)

In [15]:
start_instant = Instant.date_time(DateTime.parse("2018-01-01 00:00:00"), Scale.UTC)
end_instant = Instant.date_time(DateTime.parse("2018-01-10 00:00:00"), Scale.UTC)

interval = Interval.closed(start_instant, end_instant)

In [16]:
access_generator = AccessGenerator(environment=environment)


In [17]:
_generator = AccessGenerator(environment=environment)
accesses = access_generator.compute_accesses(
    interval=interval,
    access_targets=access_targets,
    to_trajectory=satellite_orbit,
)

assert len(accesses) == len(access_targets)  # a list of accesses per target

first_target_accesses = accesses[0]
second_target_accesses = accesses[1]
all_accesses = first_target_accesses + second_target_accesses

accesses_data = [
    (
        str(access.get_type()),
        repr(access.get_acquisition_of_signal()),
        repr(access.get_time_of_closest_approach()),
        repr(access.get_loss_of_signal()),
        float(access.get_duration().in_seconds()),
        "first_target",
    ) for access in first_target_accesses
]

accesses_data += [
    (
        str(access.get_type()),
        repr(access.get_acquisition_of_signal()),
        repr(access.get_time_of_closest_approach()),
        repr(access.get_loss_of_signal()),
        float(access.get_duration().in_seconds()),
        "second_target",
    ) for access in second_target_accesses
]

Locking local repository [/var/cache/open-space-toolkit-data/data/manifest]...
Fetching Data Manifest from [https://github.com/open-space-collective/open-space-toolkit-data/raw/v1/data/manifest.json]...
Unlocking local repository [/var/cache/open-space-toolkit-data/data/manifest]...
Data Manifest [/var/cache/open-space-toolkit-data/data/manifest/manifest.json] has been successfully fetched from [https://github.com/open-space-collective/open-space-toolkit-data/raw/v1/data/manifest.json].


In [18]:
accesses_df = pd.DataFrame(
    data=accesses_data,
    columns=["Type", "AOS", "TCA", "LOS", "Duration", "Target"],
)

In [19]:
accesses_df

,Type,AOS,TCA,LOS,Duration,Target
0,Type.Complete,2018-01-01 09:59:18.137.764.367 [UTC],2018-01-01 10:02:55.820.985.086 [UTC],2018-01-01 10:06:28.221.754.455 [UTC],430.083990,first_target
1,Type.Complete,2018-01-01 11:32:27.878.466.172 [UTC],2018-01-01 11:37:02.526.058.842 [UTC],2018-01-01 11:41:32.685.618.049 [UTC],544.807152,first_target
2,Type.Complete,2018-01-01 13:08:30.913.272.317 [UTC],2018-01-01 13:09:58.208.348.333 [UTC],2018-01-01 13:11:21.748.897.280 [UTC],170.835625,first_target
3,Type.Complete,2018-01-01 20:42:42.329.783.076 [UTC],2018-01-01 20:46:05.893.141.774 [UTC],2018-01-01 20:49:33.404.254.308 [UTC],411.074471,first_target
4,Type.Complete,2018-01-01 22:14:55.129.176.158 [UTC],2018-01-01 22:19:29.181.473.576 [UTC],2018-01-01 22:24:08.132.120.130 [UTC],553.002944,first_target
...,...,...,...,...,...,...
84,Type.Complete,2018-01-08 22:46:45.400.029.163 [UTC],2018-01-08 22:50:37.893.248.030 [UTC],2018-01-08 22:54:25.118.392.996 [UTC],459.718364,second_target
85,Type.Complete,2018-01-09 00:20:08.483.840.770 [UTC],2018-01-09 00:24:38.688.956.700 [UTC],2018-01-09 00:29:04.510.069.054 [UTC],536.026228,second_target
86,Type.Complete,2018-01-09 09:30:01.302.237.087 [UTC],2018-01-09 09:33:39.037.512.306 [UTC],2018-01-09 09:37:20.769.829.638 [UTC],439.467593,second_target
87,Type.Complete,2018-01-09 11:02:36.870.760.756 [UTC],2018-01-09 11:07:08.055.082.592 [UTC],2018-01-09 11:11:44.218.156.046 [UTC],547.347395,second_target


In [20]:
def compute_access_geometry(access):
    return [
        compute_time_lla_aer_coordinates(state, access_target.get_position(), environment)
        for state in satellite_orbit.get_states_at(
            access.get_interval().generate_grid(Duration.seconds(1.0))
        )
    ]

In [21]:
satellite_orbit_geometry_df = pd.DataFrame(
    [lla.to_vector() for lla in compute_trajectory_geometry(satellite_orbit, interval)],
    columns=["Latitude", "Longitude", "Altitude"],
)

In [22]:
satellite_orbit_geometry_df.head()

,Latitude,Longitude,Altitude
0,-0.020152,-0.001105,500000.002625
1,3.772279,-0.735253,500091.837383
2,7.564031,-1.473598,500367.676627
3,11.354418,-2.220508,500822.610781
4,15.142749,-2.980651,501448.546393


In [23]:
access_geometry_dfs = [
    pd.DataFrame(
        compute_access_geometry(access),
        columns=[
            "Time",
            "Latitude",
            "Longitude",
            "Altitude",
            "Azimuth",
            "Elevation",
            "Range",
        ],
    )
    for access in all_accesses
]

In [24]:
access_geometry_dfs

[                                Time   Latitude  Longitude       Altitude  \
 0   2018-01-01 09:59:18.137764+00:00  61.999847  44.042447  516642.310364   
 1   2018-01-01 09:59:19.137764+00:00  61.939068  44.002145  516623.407511   
 2   2018-01-01 09:59:20.137764+00:00  61.878280  43.961987  516604.474832   
 3   2018-01-01 09:59:21.137764+00:00  61.817481  43.921972  516585.512418   
 4   2018-01-01 09:59:22.137764+00:00  61.756672  43.882097  516566.520362   
 ..                               ...        ...        ...            ...   
 427 2018-01-01 10:06:25.137764+00:00  35.534442  33.615179  507185.673565   
 428 2018-01-01 10:06:26.137764+00:00  35.471758  33.598904  507163.608748   
 429 2018-01-01 10:06:27.137764+00:00  35.409072  33.582649  507141.560254   
 430 2018-01-01 10:06:28.137764+00:00  35.346385  33.566412  507119.528192   
 431 2018-01-01 10:06:28.221754+00:00  35.341119  33.565049  507117.678469   
 
         Azimuth  Elevation         Range  
 0     38.711882  